<a href="https://colab.research.google.com/github/machine-perception-robotics-group/JDLALectureNotebooks/blob/master/notebooks/17_lstm_encoder_decoder_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Encoder-Decoderによる機械翻訳

---
## 目的
Encoder-Decoder構造のネットワークである（Sequence-to-Sequenel Seq2Seq）を用いて機械翻訳（English-French）を行う．


## 対応するチャプター
* 10.4: Encoder-DecoderとSequence-to-Sequence

## データのダウンロード
実習に必要なデータをダウンロードします．
下記のコードを実行してデータのダウンロードを行ってください．

In [ ]:
!wget http://www.mprg.cs.chubu.ac.jp/~hirakawa/share/tutorial_data/fra-eng-tr_data.zip
!unzip -q -o fra-eng-tr_data.zip
!ls ./fra-eng-tr_data

## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．

In [ ]:
from time import time
import json
import numpy
import numpy as np

import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn

## データセットの読み込み
データセットを読み込みます．
今回のデータはフランス語と英語の対訳データセットを使用します．
使用するデータには，学習データとして8479，テストデータとして2120の対訳文が含まれています．

また，このデータセットに存在する単語の情報を取得します．
`fra_vocab`にはフランス語の単語情報，`eng_vocab`には英語の単語情報を含んだ辞書オブジェクトが読み込まれます．

In [ ]:
def download_text_dataset(input_filename):
    downloaded = []
    with open(input_filename) as f:
        for s in f.readlines():
            downloaded.append( np.array(list(map(int, s.strip().split(' '))), dtype=np.int32) )
    return downloaded

fra_train = download_text_dataset("fra-eng-tr_data/fra_train.txt")
eng_train = download_text_dataset("fra-eng-tr_data/eng_train.txt")
fra_test = download_text_dataset("fra-eng-tr_data/fra_test.txt")
eng_test = download_text_dataset("fra-eng-tr_data/eng_test.txt")

print(len(fra_train))
print(len(fra_test))
        
with open("fra-eng-tr_data/fra_vocab.json") as f:
    fra_vocab = json.load(f)
    fra_vocab = {int(k):v for k, v in fra_vocab.items()}
    
with open("fra-eng-tr_data/eng_vocab.json") as f:
    eng_vocab = json.load(f)
    eng_vocab = {int(k):v for k, v in eng_vocab.items()}

## ネットワークモデルの定義
Seq2Seqのネットワークを定義します．

文章の終わりを表現する単語IDとして`EOS=1`を定義します．

次に，`sequence_embed`関数を定義します．
この関数では，この次に定義するネットワーク内部での計算を行う際に，任意の長さの文章をまとめたミニバッチデータに対して，ある処理を行うための関数です．
この関数を用いてネットワークの演算を行うことで，ミニバッチ中に含まれる文章の長さが異なる場合でも，一度に処理を行うことが可能となります．

翻訳を行うための`Seq2seq`ネットワークを定義します．
このネットワークには，入力された文章（フランス語文）を処理するEncoderと翻訳結果（英文）を出力するDecoderが存在します．

まず`__init__`関数でネットワーク内部の層を定義します．
`__init__`関数の引数として，次の引数を準備します．
`n_layers`はEncoderおよびDecoderに用いるLSTMの総数，`n_source_vocab`は入力側で扱う単語数，`n_target_vocab`は出力側で扱う単語数，`n_units`はLSTMの隠れ層のユニット数です．
`embed_x`および`embed_y`ではそれぞれ，入力・出力側で扱う単語のIDから単語を表現した特徴ベクトルを出力するための`Embedding`層を定義します．
この`Embedding`層に入る単語IDで埋め込みベクトルを計算したくないもの(例えばパディングしたID)を`padding_idx`で設定できます．
次に，`encoder`および`decoder`では，LSTM層を定義します．
ここでは，`nn.LSTM`という層を使用して定義を行います．
`nn.LSTM`は，文章の長さを揃え，ミニバッチを同時に扱うことができるLSTMの定義方法です．
ここでは，LSTMの層数やdropoutのdrop率も同時に定義することが可能です．
しかし，`nn.LSTM`はミニバッチ内で文章の長さが統一されてない場合に処理できません．
そこで，`pad_sequence`でミニバッチ内で文章の長さを揃えるようにします．
`pad_sequence`はミニバッチ内で最も文章の長さに合わせて短い文章の末尾にパディングを追加し次元を揃えてくれる関数になります．
このパディングは任意の数に設定できます．今回は0で設定します．
最後に，`decoder`から出力された特徴を元に翻訳結果の単語IDを出力するための出力層`W`を定義します．


次に，入力されたデータから結果を出力するための演算を定義します．
ここでは，下記で実行する学習のプログラムを簡単にするため，2種類の演算の関数を定義します．
`forward`関数では，学習を行う際の演算を定義します．ネットワークの出力結果と正解の翻訳情報から計算した誤差を返す関数として定義します．
また，`translate`関数では，ネットワークが予測した翻訳結果を出力するように定義します．

In [ ]:
EOS = 1

def sequence_embed(embed, xs):
    x_len = [len(x) for x in xs]
    x_section = x_len
    ex = embed(torch.cat(xs, dim=0))
    exs = torch.split(ex, x_section, 0)
    return exs

class Seq2seq(nn.Module):

    def __init__(self, n_layers, n_source_vocab, n_target_vocab, n_units):
        super(Seq2seq, self).__init__()
        self.embed_x = nn.Embedding(n_source_vocab, n_units, padding_idx=0)
        self.embed_y = nn.Embedding(n_target_vocab, n_units, padding_idx=0)
        self.encoder = nn.LSTM(n_units, n_units, num_layers=n_layers, batch_first=True, dropout=0.1)
        self.decoder = nn.LSTM(n_units, n_units, num_layers=n_layers, batch_first=True, dropout=0.1)
        self.W = nn.Linear(n_units, n_target_vocab)

        self.n_layers = n_layers
        self.n_units = n_units

    def forward(self, xs, ys, criterion):
        xs = [torch.flipud(x) for x in xs]

        eos = torch.from_numpy(np.array([EOS], np.int64)).cuda()
        ys_in = [torch.cat([eos, y], dim=0) for y in ys]
        ys_out = [torch.cat([y, eos], dim=0) for y in ys]

        exs = sequence_embed(self.embed_x, xs)
        eys = sequence_embed(self.embed_y, ys_in)

        batch = len(xs)
        packed_encode_input = rnn.pad_sequence(exs, batch_first=True, padding_value=0)
        packed_decode_input = rnn.pad_sequence(eys, batch_first=True, padding_value=0)
        packed_target = rnn.pad_sequence(ys_out, batch_first=True, padding_value=0)

        e_output, e_hidden = self.encoder(packed_encode_input)
        d_output, d_hidden = self.decoder(packed_decode_input, e_hidden)

        output = self.W(d_output)
        b, s, c = output.shape
        output = output.reshape(b*s, c)
        packed_target = packed_target.reshape(b*s)

        loss = criterion(output, packed_target)
        return loss

    def translate(self, xs, max_length=100):
        batch = len(xs)
        with torch.autograd.no_grad():
            xs = [torch.flipud(x) for x in xs]
            exs = sequence_embed(self.embed_x, xs)
            packed_encode_input = rnn.pad_sequence(exs, batch_first=True, padding_value=0)
            e_output, hidden = self.encoder(packed_encode_input)
            ys = torch.from_numpy(np.array([[EOS]], np.int64)).cuda()
            result = []
            for i in range(max_length):
                eys = self.embed_y(ys) 
                d_output, hidden = self.decoder(eys, hidden)
                d_output = d_output.view(1, d_output.shape[-1])
                output = self.W(d_output)
                ys = torch.argmax(output, dim=-1, keepdim=True)
                output = torch.argmax(output, dim=-1)
                result.append(output)

        return result

## ネットワークの作成
上のプログラムで定義したネットワークを作成します．
ここでは，GPUで学習を行うために，modelをGPUに送るcuda()関数を利用しています．

学習を行う際の最適化方法としてモーメンタムSGD(モーメンタム付き確率的勾配降下法）を利用します．また，学習率を0.001として引数に与えます．

In [ ]:
num_source_vocab = len(fra_vocab)
num_target_vocab = len(eng_vocab)
model = Seq2seq(n_layers=2,
                n_source_vocab=num_source_vocab,
                n_target_vocab=num_target_vocab,
                n_units=1024)
model.cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## 学習

学習を実行します．
`nn.CrossEntropyLoss`内の`ignore_index`は指定したクラスインデックスを計算しないようにする引数です．
今回，ミニバッチ内で文章の長さを揃えるためにパディングを行っており，そのまま計算するとノイズになり得るため，余分なクラスインデックスを計算しないようにします．

In [ ]:
# ミニバッチサイズ・エポック数．学習データ数の設定
batch_size = 32
epoch_num = 100
train_data_num = len(fra_train)
num_iter_per_epoch = int(train_data_num / batch_size)

# 誤差関数の設定
criterion = nn.CrossEntropyLoss(ignore_index=0)
criterion.cuda()

model.train()
# 学習の実行
start = time()
for epoch in range(1, epoch_num + 1):
    
    sum_loss = 0
    
    perm = np.random.permutation(train_data_num)
    for i in range(0, train_data_num, batch_size):
        optimizer.zero_grad()
        fra_batch = [torch.from_numpy(np.array(fra_train[ii], dtype=np.int64)).cuda() for ii in perm[i:i+batch_size] ]
        eng_batch = [torch.from_numpy(np.array(eng_train[ii], dtype=np.int64)).cuda() for ii in perm[i:i+batch_size] ]

        loss = model(fra_batch, eng_batch, criterion)        
        
        sum_loss += loss.data

        loss.backward()
        optimizer.step()

    elapsed_time = time() - start
    print("epoch: {}, mean loss: {}, elapsed_time: {}".format(epoch,
                                                              sum_loss/num_iter_per_epoch,
                                                              elapsed_time))

    if epoch % 20 == 0:
        model_name = "model_v{}.pt".format(epoch)
        torch.save({'model': model.state_dict()}, model_name)

## テスト

学習後のネットワークを用いて，翻訳を行います．

In [ ]:
model.load_state_dict(torch.load("model_v300.pt")['model'])
model.cuda()
model.eval()
with torch.autograd.no_grad():
    test_index = np.arange(20)
    
    for i_test in test_index:
        fra_batch = [ torch.from_numpy(np.array(fra_test[i_test], dtype=np.int64)).cuda() ]
        eng_batch = [ torch.from_numpy(np.array(eng_test[i_test], dtype=np.int64)).cuda() ]
    
        y = model.translate(fra_batch, max_length=10)

        fra_true = [fra_vocab[i] for i in fra_batch[0].detach().cpu().numpy()]
        eng_pred = [eng_vocab[i.detach().cpu().numpy()[0]] for i in y]

        print("input french sentence      :", " ".join(fra_true))
        print("translated english sentence:", " ".join(eng_pred), "\n")